In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
from pandas import Series
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
# Import the data
import json
dma = pd.read_csv("C:/Users/user/Desktop/Rutgers/DMA-zip.csv", encoding="iso-8859-1")
events = json.load(open('C:/Users/user/Desktop/Rutgers/events.json', 'r'))
#Convert list "event" to dataframe
events_df = pd.DataFrame(events)

In [3]:
# First look at the data
dma.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41535 entries, 0 to 41534
Data columns (total 6 columns):
FIPS        41535 non-null int64
COUNTY      41535 non-null object
ST          41535 non-null object
DMA CODE    41535 non-null int64
DMA NAME    41535 non-null object
ZIPCODE     41535 non-null int64
dtypes: int64(3), object(3)
memory usage: 1.9+ MB


In [4]:
events_df.info()
events_df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81474 entries, 0 to 81473
Data columns (total 16 columns):
event_description    81474 non-null object
event_end_utc        81474 non-null object
event_id             81474 non-null object
event_name           81474 non-null object
event_start_utc      81474 non-null object
facebook_event_id    53788 non-null object
hashtag              14287 non-null object
organization_id      81474 non-null object
organization_name    81474 non-null object
tags                 81474 non-null object
venue_city           81474 non-null object
venue_name           81474 non-null object
venue_state          81474 non-null object
venue_street         81474 non-null object
venue_timezone       81474 non-null object
venue_zip            81474 non-null object
dtypes: object(16)
memory usage: 9.9+ MB


Index(['event_description', 'event_end_utc', 'event_id', 'event_name',
       'event_start_utc', 'facebook_event_id', 'hashtag', 'organization_id',
       'organization_name', 'tags', 'venue_city', 'venue_name', 'venue_state',
       'venue_street', 'venue_timezone', 'venue_zip'],
      dtype='object')

In [5]:
# Filling missing values with "Unknown"
events_df=events_df.fillna(value="Unknown")

In [6]:
events_df.head()

,event_description,event_end_utc,event_id,event_name,event_start_utc,facebook_event_id,hashtag,organization_id,organization_name,tags,venue_city,venue_name,venue_state,venue_street,venue_timezone,venue_zip
0,Moto stickers\n,17 Jun 2017 18:30:00,16a82bf8-516a-11e7-a10b-22000bb0aa3e,Motobrewtour,17 Jun 2017 18:00:00,,Unknown,campbell-inc,Campbell inc,[],Salisbury,1623 Crawford Dr,MD,1623 Crawford Dr,US/Eastern,21804
1,"Explore a 6-acre Maze, visit the Pumpkin Patch...",12 Nov 2017 23:00:00,4559a156-7669-11e7-ab7f-22000b238d3d,Rockin' K Maze,30 Sep 2017 16:00:00,,rockinkmaze2017,rockinkfarms,Rockin' K Farms,[],Robstown,Rockin' K Maze,TX,5385 CR 52,US/Central,78380
2,Celebrate Halloween Regency style with this sc...,27 Oct 2018 01:00:00,48414fde-ac70-11e8-9d04-22000ab816c3,Outdoor Movie Night: Pride and Prejudice and Z...,26 Oct 2018 21:30:00,,Unknown,dumbartonhouse,Dumbarton House,[],Washington,Dumbarton House,DC,2715 Q St NW,US/Eastern,20007
3,"Family Friendly Wrestling in Muskegon, MI feat...",20 Nov 2016 02:00:00,cff666d4-85d1-11e6-9cdc-22000ae18222,PPW LIVE IN MUSKEGON,19 Nov 2016 23:00:00,,Unknown,prowrestlingatitsfinest,Pure Pro Wrestling,[],Muskegon,Muskegon VFW Post 446,MI,165 S Getty St,US/Eastern,49442
4,Come join members of theÂ Empower Texans staff...,18 Jan 2018 02:00:00,b8f8d1e2-f713-11e7-95bb-22000a66e11f,Dallas County Happy Hour,18 Jan 2018 00:00:00,Unknown,Unknown,empowertexans,Empower Texans,[],Dallas,Yard House - Addison,TX,5100 Belt Line Rd,US/Central,75254


In [7]:
events_df_new = events_df[['event_description', 'event_name','organization_name','venue_city', 'venue_state']]

In [54]:
#Create a new column called info which will contain event description; event name; state within each record:
events_df_new['info'] = events_df_new['event_description']+' '+events_df_new['event_name']+' '+events_df_new['venue_state'] 
#+ events_df_new['organization_name']
events_df_new['info'].fillna('', inplace=True)
events_df_new.iloc[2]

event_description    Celebrate Halloween Regency style with this sc...
event_name           Outdoor Movie Night: Pride and Prejudice and Z...
organization_name                                      Dumbarton House
venue_city                                                  Washington
venue_state                                                         DC
info                 Celebrate Halloween Regency style with this sc...
Name: 2, dtype: object

In [55]:
#TfidfVectorizer function from scikit-learn, which transforms text to feature vectors that can be used as input to estimator.

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist


tfidf = TfidfVectorizer(input='content', encoding='iso-8859-1', 
                      decode_error='ignore', analyzer='word', max_df=0.8, # drop words that occur in more than 80% of articles
                      min_df=5, # only use words that appear in at least 5 articles 
                      lowercase=True, #convert everything to lower case
                      ngram_range=(1,2), stop_words='english')

In [56]:
tfidf_m = tfidf.fit_transform(events_df_new['info'])
print("Number of features: %d" % tfidf_m.get_shape()[1])
tfidf_m.shape

Number of features: 242281


(81474, 242281)

In [11]:
type(tfidf_m)

scipy.sparse.csr.csr_matrix

In [57]:
#Compute the SVD and convert data to latent space

import scipy
from scipy import linalg
from scipy.sparse.linalg import svds
from sklearn.metrics.pairwise import linear_kernel

U, sigma, Vt = svds(tfidf_m, k= 100)
U.shape

(81474, 100)

In [39]:
U[0]

array([-2.42205913e-04,  1.09274348e-03,  1.51066140e-03,  1.07038821e-03,
       -5.53576591e-05,  1.86006851e-03,  6.52687892e-04,  7.98865203e-05,
       -7.01324637e-04,  6.54143530e-04, -1.99934170e-04,  1.68981203e-04,
       -1.18862894e-03,  9.96808383e-04, -2.74618807e-04,  4.47283953e-04,
        7.44931615e-04, -8.63038318e-04, -6.00309657e-04,  3.88547987e-04,
        8.32715605e-04,  4.17046084e-04, -3.57503824e-04,  5.87275281e-04,
        7.93484204e-04, -6.70976079e-04, -9.57511939e-05,  3.04249012e-04,
       -5.03867627e-04, -4.61884143e-04, -7.42249199e-05,  4.09843478e-04,
        6.98955519e-04, -1.92501069e-03, -3.15246861e-04,  4.74715475e-04,
        8.39877891e-05, -6.14244915e-04, -5.66166261e-04, -1.10552181e-04,
       -1.07550883e-04, -2.34519364e-04,  7.26185073e-04, -1.61960349e-04,
        2.11777050e-03, -3.39364670e-03,  1.63964361e-03, -2.60070091e-03,
       -2.49467881e-03,  2.67904709e-03,  5.58937365e-03,  1.37613283e-03,
        1.86541352e-04,  

I will be using the Cosine Similarity to calculate the similarity between two events. 
Since we have used the TF-IDF Vectorizer, calculating the Dot Product will directly give us the Cosine Similarity Score. 

In [71]:
#The next step is to write a function that returns the 5 most similar events based on the cosine similarity score.

event_name = events_df_new['event_name']
indices = pd.Series(events_df_new.index, index=events_df_new['event_name'])

from sklearn.metrics.pairwise import cosine_similarity
def events_recommendations(r):
    idx = indices[r]
    cos = cosine_similarity(U[idx:int(idx)+1], U)
    sim_scores = list(enumerate(cos.ravel()))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    event_indices = [i[0] for i in sim_scores]
    pd.set_option('display.max_colwidth', -1)
    return events_df[['event_name','venue_street','venue_city', 'venue_state','venue_zip','event_description']].iloc[event_indices]

In [72]:
#Testing
#pd.set_option('display.max_colwidth', -1)
events_recommendations('PPW LIVE IN MUSKEGON')


,event_name,venue_street,venue_city,venue_state,venue_zip,event_description
39083,PPW LIVE IN SUMNER,10820 Pine St,Sumner,MI,48889,"Family Friendly Wrestling in Sumner, MI featuring WWE Alumni and NBC American Ninja Warrior Zach Gowen!\n"
68582,PPW LIVE IN GRANT,105 S Front St,Grant,MI,49327,"Family Friendly Wrestling comes to Grant, MI!\n"
54500,PPW LIVE IN FLUSHING,5380 W Pierson Rd,Flushing,MI,48433,"PPW brings family friendly pro wrestling to Flushing, MI!\n"
32078,PPW LIVE IN UNIONVILLE,2960 Cass St,Unionville,MI,48767,Pure Pro Wrestling brings family friendly professional wrestling to Unionville! WWE Alumni Zach Gowen leads an all star cast of grapplers as they invade the squared circle.\n
12191,PPW Live in Manistee,1211 28th St,Manistee,MI,49660,Pure Pro Wrestling Returns to Manistee VFW for another action packed night of family friendly pro wrestling! WWE Alumni Zach Gowen leads an all star cast of grapplers as they invade the squared circle\n
